# PLCTestbench

In [ ]:
from plctestbench.settings import *
from plctestbench.plc_testbench import PLCTestbench
from plctestbench.loss_simulator import BinomialPLS, GilbertElliotPLS
from plctestbench.plc_algorithm import ZerosPLC, LastPacketPLC, LowCostPLC, BurgPLC, ExternalPLC
from plctestbench.output_analyser import MSECalculator, MAECalculator, PEAQCalculator
from plctestbench.worker import OriginalAudio

testbench_settings = {
    'root_folder': '../original_tracks',
    'db_ip': '127.0.0.1',
    'db_username': 'myUserAdmin',
    'db_password': 'admin',
}

original_audio_tracks = [(OriginalAudio, OriginalAudioSettings('Blues_Guitar.wav')),
                         (OriginalAudio, OriginalAudioSettings('Blues_Piano.wav'))]

packet_loss_simulators = [(GilbertElliotPLS, GilbertElliotPLSSettings()),
                          (BinomialPLS, BinomialPLSSettings())]

crossfade_settings = [QuadraticCrossfadeSettings(length=50), QuadraticCrossfadeSettings(length=5), QuadraticCrossfadeSettings(length=5)]
frequencies = [200, 2000]
plc_algorithms = [(ZerosPLC, ZerosPLCSettings(frequencies = frequencies, fade_in=[QuadraticCrossfadeSettings(length=10)], crossfade=crossfade_settings)),
                  (LastPacketPLC, LastPacketPLCSettings(frequencies = frequencies, fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=crossfade_settings, mirror_x=True, mirror_y=True)),
                  (LowCostPLC, LowCostPLCSettings(frequencies = frequencies, fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=crossfade_settings)),
                  #(BurgPLC, BurgPLCSettings(frequencies = frequencies, fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=crossfade_settings)),
                  (ExternalPLC, ExternalPLCSettings())
                  ]
metrics = [(MSECalculator, MSECalculatorSettings()),
           (MAECalculator, MAECalculatorSettings())
           #,(PEAQCalculator, PEAQCalculatorSettings())
           ]

testbench = PLCTestbench(original_audio_tracks, packet_loss_simulators, plc_algorithms, metrics, testbench_settings)

In [2]:
testbench.run()

In [ ]:
%matplotlib inline
testbench.plot(to_file=True, original_tracks=True, lost_samples_masks=True, output_analyses=True)